In [17]:
from modules.calculations import calcular_fatura_cativa, calcular_fatura_uso,calcular_fatura_livre
from modules.data_utils import get_tariffs
from modules.plot_generator import energy_cost_plot

In [4]:
quantidade = {
    "Demanda HFP": 100, 
    "Demanda HFP sICMS": 0,  
    "Demanda HP": 0,    
    "Demanda HP sICMS": 0,  
    "Demanda HR": 0,    
    "Demanda HR sICMS": 0,  
    "Energia HFP": 10000, 
    "Energia HP": 1000,   
    "Energia HR": 0,   
    "Energia Compensada HFP": 0, 
    "Energia Compensada HP": 0 
}

icms = 18
icms_hr = 0
desc_irrig = 0.8
cofins = 3.82
paseb = 0.83
bandeira = "verde"
impostos_bandeira = {
    "icms": icms/100,
    "paseb": paseb/100,
    "cofins": cofins/100,
    "bandeira": bandeira,
    "icms_hr": icms_hr,  
    "desc_irr": desc_irrig
}
distribuidora = "CEMIG-D"
subgrupo = "A4"
modalidade = "Verde"
resolucao = "RESOLUÇÃO HOMOLOGATÓRIA Nº 3.328, DE 21 DE MAIO DE 2024"

tarifa = get_tariffs(distribuidora, subgrupo, modalidade, resolucao)

produto = "PMT"
desconto = 0

preco = {
    "preco": [263.38, 263.38, 263.38, 263.38, 263.38, 263.38],
    "produto": produto,
    "anos": [2026, 2027, 2028, 2029, 2030, 2031],
    "duracao_meses": 60,
    "desconto": desconto/100
}


fatura_cativa = calcular_fatura_cativa(quantidade, tarifa, impostos_bandeira)
fatura_uso = calcular_fatura_uso(quantidade, tarifa, impostos_bandeira)
fatura_livre = calcular_fatura_livre(quantidade, preco, impostos_bandeira, fatura_uso, fatura_cativa)

2025-04-15 10:37:19.749 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-15 10:37:19.751 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-15 10:37:19.754 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-15 10:37:19.761 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-15 10:37:19.763 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-15 10:37:19.765 No runtime found, using MemoryCacheStorageManager
2025-04-15 10:37:19.768 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-15 10:37:19.769 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-15 10:37:19.772 Thread 'MainThread':

calcular_fatura_livre - fatura_cativa: {'Demanda HFP': 2843.183649455792, 'Demanda HFP sICMS': 0.0, 'Demanda HP': 0.0, 'Demanda HP sICMS': 0.0, 'Energia HFP': 5379.9224935091515, 'Energia HP': 2813.0891324644763, 'Energia HR': 0.0, 'Energia Compensada HFP': 0.0, 'Energia Compensada HP': 0.0, 'Desconto Irrigante Noturno': 0.0, 'Custo da Bandeira': 0.0, 'Fatura Cativa': 11036.19527542942}
calcular_fatura_livre - fatura_uso: {'Demanda HFP': 2843.183649455792, 'Demanda HFP sICMS': 0.0, 'Demanda HP': 0.0, 'Demanda HP sICMS': 0.0, 'Energia HFP': 1572.5120544336014, 'Energia HP': 2213.9997697827002, 'Energia HR': 0.0, 'Energia Compensada HFP': None, 'Energia Compensada HP': None, 'Desconto Irrigante Noturno': 0.0, 'Desconto Demanda HFP': 1111.5, 'Desconto Demanda HP': 0.0, 'Desconto TUSD HP': 804.055, 'Fatura de Uso': 4714.140473672093}
calcular_fatura_livre - len(preco['anos']): 6
calcular_fatura_livre - len(preco['preco']): 6
calcular_fatura_livre - quantidade: {'Demanda HFP': 100, 'Demanda

2025-04-15 10:37:19.798 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-15 10:37:19.800 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [5]:
fatura_cativa

{'Demanda HFP': 2843.183649455792,
 'Demanda HFP sICMS': 0.0,
 'Demanda HP': 0.0,
 'Demanda HP sICMS': 0.0,
 'Energia HFP': 5379.9224935091515,
 'Energia HP': 2813.0891324644763,
 'Energia HR': 0.0,
 'Energia Compensada HFP': 0.0,
 'Energia Compensada HP': 0.0,
 'Desconto Irrigante Noturno': 0.0,
 'Custo da Bandeira': 0.0,
 'Fatura Cativa': 11036.19527542942}

In [6]:
fatura_uso

{'Demanda HFP': 2843.183649455792,
 'Demanda HFP sICMS': 0.0,
 'Demanda HP': 0.0,
 'Demanda HP sICMS': 0.0,
 'Energia HFP': 1572.5120544336014,
 'Energia HP': 2213.9997697827002,
 'Energia HR': 0.0,
 'Energia Compensada HFP': None,
 'Energia Compensada HP': None,
 'Desconto Irrigante Noturno': 0.0,
 'Desconto Demanda HFP': 1111.5,
 'Desconto Demanda HP': 0.0,
 'Desconto TUSD HP': 804.055,
 'Fatura de Uso': 4714.140473672093}

In [7]:
fatura_livre

{'Fatura Livre': [3533.1463414634145,
  3533.1463414634145,
  3533.1463414634145,
  3533.1463414634145,
  3533.1463414634145,
  3533.1463414634145],
 'anos': [2026, 2027, 2028, 2029, 2030, 2031]}

In [8]:
# Calculate various invoices
fatura_cativa = calcular_fatura_cativa(quantidade, tarifa, impostos_bandeira)["Fatura Cativa"]
fatura_uso = calcular_fatura_uso(quantidade, tarifa, impostos_bandeira)["Fatura de Uso"]
fatura_livre = calcular_fatura_livre(quantidade, preco, impostos_bandeira, fatura_uso, fatura_cativa)["Fatura Livre"]

# Calculate savings
economia_mensal = fatura_cativa - fatura_uso - fatura_livre[0]
economia_anual = economia_mensal * 12

2025-04-15 10:40:58.260 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-15 10:40:58.260 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-15 10:40:58.263 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


calcular_fatura_livre - fatura_cativa: 11036.19527542942
calcular_fatura_livre - fatura_uso: 4714.140473672093
calcular_fatura_livre - len(preco['anos']): 6
calcular_fatura_livre - len(preco['preco']): 6
calcular_fatura_livre - quantidade: {'Demanda HFP': 100, 'Demanda HFP sICMS': 0, 'Demanda HP': 0, 'Demanda HP sICMS': 0, 'Demanda HR': 0, 'Demanda HR sICMS': 0, 'Energia HFP': 10000, 'Energia HP': 1000, 'Energia HR': 0, 'Energia Compensada HFP': 0, 'Energia Compensada HP': 0}
calcular_fatura_livre - fatura_livre: [3533.1463414634145, 3533.1463414634145, 3533.1463414634145, 3533.1463414634145, 3533.1463414634145, 3533.1463414634145]


2025-04-15 10:40:58.273 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-15 10:40:58.275 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [9]:
economia_anual

33466.90152352695

In [10]:
economia_mensal

2788.908460293912

In [12]:
months = [min(12, max(0, preco["duracao_meses"] - 12*i)) for i in range(len(preco["anos"]))]
economy = [(fatura_cativa - fatura_uso - fatura_livre[i])*months[i] for i in range(len(months))]
percentual_economy = [(fatura_cativa - fatura_uso - fatura_livre[i])/fatura_cativa for i in range(len(fatura_livre))]
    

In [13]:
months

[12, 12, 12, 12, 12, 0]

In [14]:
economy

[33466.90152352695,
 33466.90152352695,
 33466.90152352695,
 33466.90152352695,
 33466.90152352695,
 0.0]

In [15]:
percentual_economy

[0.2527056101030611,
 0.2527056101030611,
 0.2527056101030611,
 0.2527056101030611,
 0.2527056101030611,
 0.2527056101030611]

In [19]:
energy_cost_plot(fatura_cativa,fatura_livre[0], fatura_uso, economy[0]/12)


Light bulb image loaded successfully: (73, 253, 3)
Power line image loaded successfully: (93, 289, 3)
